In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Softmax, Input, BatchNormalization, Dropout

from sklearn.model_selection import train_test_split

import tensorflow as tf
from load_policy import load_policy
import pickle
import numpy as np
import gym

import tf_util


Using TensorFlow backend.


In [3]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

In [6]:
expert_policy_file = "experts/HalfCheetah-v1.pkl"
env_name = "HalfCheetah-v1"
rounds = 1000
max_steps = 1000

In [7]:
data = pickle.load(open(expert_policy_file, "rb"))
policy_fn = load_policy(expert_policy_file)

obs (1, 17) (1, 17)


In [8]:
observations = []
actions = []
rewards = []

with tf.Session() as sess:
#     tf_util.initialize()
    env = gym.make(env_name)
    for i in range(rounds):
        obs = env.reset()
        r = 0
        for s in range(max_steps):
            action = policy_fn(obs[None,:])
            observations.append(obs)
            actions.append(action)
            obs, r_, done, _ = env.step(action)
            rewards.append(r_)
            r += r_
            if done:
                break
        if i and i%100 == 0:
            print(i)
            

[2018-06-06 12:50:22,511] Making new env: HalfCheetah-v1


100
200
300
400
500
600
700
800
900


In [9]:
observations = np.array(observations)
actions = np.vstack(actions)
rewards = np.array(rewards)

obs_train, obs_valid, act_train, act_valid = \
    train_test_split(observations, actions, test_size = 0.2, random_state = 1998)

In [10]:
actions.shape

(1000000, 6)

In [14]:
input_dim = observations.shape[-1]
bc = Sequential()
bc.add(Dense(32, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
bc.add(Dense(128, input_dim=input_dim, kernel_initializer='normal', activation='relu'))

bc.add(Dense(512, kernel_initializer='normal', activation='relu'))
# bc.add(BatchNormalization())
# bc.add(Dropout(0.5))
bc.add(Dense(128, kernel_initializer='normal', activation='relu'))

bc.add(Dense(6, kernel_initializer='normal'))


In [15]:
bc.compile(optimizer='adam',loss='mse', metrics=['accuracy', 'mae'])

In [ ]:
bc.fit(obs_train, act_train, validation_data=[obs_valid, act_valid],  epochs=20, batch_size= 512)

Train on 800000 samples, validate on 200000 samples
Epoch 1/20
800000/800000 [==============================] - 3s 4us/step - loss: 0.0167 - acc: 0.9060 - mean_absolute_error: 0.0711 - val_loss: 0.0027 - val_acc: 0.9414 - val_mean_absolute_error: 0.0372
Epoch 2/20
800000/800000 [==============================] - 3s 4us/step - loss: 0.0015 - acc: 0.9603 - mean_absolute_error: 0.0275 - val_loss: 0.0010 - val_acc: 0.9692 - val_mean_absolute_error: 0.0232
Epoch 3/20
800000/800000 [==============================] - 3s 4us/step - loss: 8.6379e-04 - acc: 0.9688 - mean_absolute_error: 0.0213 - val_loss: 0.0011 - val_acc: 0.9592 - val_mean_absolute_error: 0.0237
Epoch 4/20
800000/800000 [==============================] - 3s 4us/step - loss: 6.3799e-04 - acc: 0.9729 - mean_absolute_error: 0.0184 - val_loss: 5.4850e-04 - val_acc: 0.9733 - val_mean_absolute_error: 0.0172
Epoch 5/20
800000/800000 [==============================] - 3s 4us/step - loss: 5.1200e-04 - acc: 0.9759 - mean_absolute_error: 

In [19]:
rewards.sum()/1000

4139.245206793351

In [20]:
o = np.random.random(17)[None, :]
bc.predict(o)

array([[-1.4889042 , -0.15402749, -0.26528874, -0.7841745 ,  0.13741665,
         1.2115028 ]], dtype=float32)

In [21]:
with tf.Session():
    print(policy_fn(o))

[[-1.7382091  -0.3162961  -0.05374527 -0.9472675   0.22024074  1.1463321 ]]


In [26]:
m_rewards = []

def run_model(m, m_rounds, max_steps):
    env = gym.make(env_name)
    for i in range(m_rounds):
        obs = env.reset()
        r = 0
        for s in range(max_steps):
            action = m.predict(obs[None,:])
#             observations.append(obs)
#             actions.append(action)
            obs, r_, done, _ = env.step(action)
            m_rewards.append(r_)
#             r += r_
            if done:
                break

In [ ]:
mr = 1000
mstep = 1000
run_model(bc, mr, mstep)

[2018-06-06 13:16:52,750] Making new env: HalfCheetah-v1


In [31]:
sum(m_rewards)/mr

4146.06614365795

In [32]:
len(m_rewards)

1000000

In [30]:
2

2